In [1]:
import torch
import torch.nn as nn
from torchvision import models

from Train.train_one_epoch import eval_epoch_save

In [2]:
model = models.densenet121()
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 2))

In [3]:
names = ['features.denseblock1', 'features.denseblock2', 'features.denseblock3', 'features.denseblock4', 'classifier']

In [4]:
loaded = torch.load('/home/zoe/GhassanGT Dropbox/Zoe Fowler/Zoe/InSync/PhDResearch/Code/Federated-SuperResolution/Saved/pneumonia/classification/model.pth')
model.load_state_dict(loaded)

<All keys matched successfully>

In [5]:
model = model.cuda()


In [ ]:
def get_gradients(data_loader, criterion, device='cuda'):
    
    loss = criterion()

    # backward pass (compute gradients of parameters w.r.t. loss)
    loss.backward()

    layer_grads = {}
    for pref in names:
        layer_grads[pref] = []

    for name, param in sam.named_parameters():
        for pref in names:
            if name.startswith(pref) and param.grad is not None:
                layer_grads[pref].append(np.linalg.norm(param.grad.detach().cpu().numpy()))

    for pref in names:
        if len(layer_grads[pref]) > 0:
            layer_grads[pref] = np.mean(layer_grads[pref])

    zero_out_grads()

    return layer_grads

In [ ]:
from Models.SRResNet import SRResNet
from torch.utils.data import DataLoader
from Data.medmnist_loader import MedMNISTDataset

# Try scale x 4 on model for x 8 and x 4
# load data and create test loader
first = 56
scale_first = int(224 / 56)
test_dataset1 = MedMNISTDataset(mode='test', low_size=first)
test_loader1 = DataLoader(test_dataset1, batch_size=1, shuffle=False)
# establish models
model1 = SRResNet(scaling_factor=4)
model2 = SRResNet(scaling_factor=8)
device = 'cuda'
wts1 = torch.load('/home/zoe/GhassanGT Dropbox/Zoe Fowler/Zoe/InSync/PhDResearch/Code/Federated-SuperResolution/Saved/pneumonia/best_model_4_resnet.pth')
wts2 = torch.load('/home/zoe/GhassanGT Dropbox/Zoe Fowler/Zoe/InSync/PhDResearch/Code/Federated-SuperResolution/Saved/pneumonia/best_model_8_resnet.pth')
model1.load_state_dict(wts1, strict=False)
model2.load_state_dict(wts2, strict=False)
model1 = model1.to(device)
model2 = model2.to(device)
criterion = nn.L1Loss().to(device)

print('Evaluating in distribution')
_, acc, outputs_indistrib = eval_epoch_save(data_loader=test_loader1, model=model1, criterion=criterion, device=device)
print('test psnr: ', acc)
# form new dataset and loader of outputs

print('Evaluating out of distribution')
_, acc, outputs_outdistrib = eval_epoch_save(data_loader=test_loader1, model=model2, criterion=criterion, device=device)
print('test psnr: ', acc)